In [1]:
import numpy as np
import gym
import random
import time
import torch
from IPython.display import clear_output
from collections import Counter
from functools import reduce
import matplotlib.pyplot as plt
#env = gym.make("FrozenLake-v0")

In [2]:
P = gym.make("FrozenLake-v0").env.P

In [3]:
UP, DOWN, LEFT, RIGHT = range(4)

In [11]:
pi = lambda s: {
    0:LEFT, 1:LEFT, 2:LEFT, 3:UP, 4:LEFT,
    5:LEFT, 6:LEFT, 7:LEFT, 8:UP, 9:LEFT,
    10:LEFT, 11:LEFT, 12:LEFT, 13:LEFT, 14:LEFT,
    15:LEFT
}[s] #the added[s] makes pi(the dictionary) callable, takes key as argument and returns value

In [12]:
P[8][1]

[(0.3333333333333333, 8, 0.0, False),
 (0.3333333333333333, 12, 0.0, True),
 (0.3333333333333333, 9, 0.0, False)]

In [13]:
import numpy as np
def policy_evaluation(pi, P, gamma = 1.0, theta=1e-10):
    prev_V = np.zeros(len(P))
    while True:
        V = np.zeros(len(P))
        for s in range(len(P)):
            for prob, next_state, reward, done in P[s][pi(s)]:
                V[s]+=prob*(reward + gamma * prev_V[next_state] * (not done))
        if np.max(np.abs(prev_V - V)) < theta:
            break
        prev_V = V.copy()
    return V

In [14]:
def policy_improvement(V, P, gamma = 1.0):
    Q_table = np.zeros((len(P), len(P[0])))
    print(Q_table.shape)
    for s in range(len(P)):
        for a in range(len(P[s])):
            for probs, next_state, reward, done in P[s][a]:
                Q_table[s][a]+=probs*(reward + gamma*V[next_state]*(not done))
    new_pi = lambda s : {s:a for s, a in enumerate(np.argmax(Q_table, axis=1))}[s]
    return new_pi,Q_table
                

In [15]:
pol_ = policy_evaluation(pi, P, gamma = 1.0, theta=1e-10)
pol_

array([0.02155172, 0.03448276, 0.06896552, 0.03448276, 0.00862069,
       0.        , 0.10344828, 0.        , 0.00431034, 0.22068966,
       0.24137931, 0.        , 0.        , 0.42068966, 0.62068966,
       0.        ])

In [16]:
new_pi, qt_ = policy_improvement(pol_, P, gamma = 1.0)

(16, 4)


In [17]:
qt_

array([[0.01724138, 0.02155172, 0.02155172, 0.02586207],
       [0.01867816, 0.03017241, 0.03448276, 0.04166667],
       [0.06896552, 0.05747126, 0.06896552, 0.04597701],
       [0.03448276, 0.03448276, 0.02298851, 0.04597701],
       [0.01149425, 0.00431034, 0.00862069, 0.01005747],
       [0.        , 0.        , 0.        , 0.        ],
       [0.10344828, 0.08045977, 0.10344828, 0.02298851],
       [0.        , 0.        , 0.        , 0.        ],
       [0.00431034, 0.075     , 0.07643678, 0.07787356],
       [0.14166667, 0.22212644, 0.22068966, 0.08189655],
       [0.31494253, 0.28045977, 0.24137931, 0.10804598],
       [0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        ],
       [0.2137931 , 0.34712644, 0.42068966, 0.28045977],
       [0.42758621, 0.68045977, 0.62068966, 0.55402299],
       [0.        , 0.        , 0.        , 0.        ]])

In [22]:
np.argmax(qt_, axis=1)

array([3, 3, 0, 3, 0, 0, 0, 0, 3, 1, 0, 0, 0, 2, 1, 0], dtype=int64)

In [18]:
qt_.shape

(16, 4)

In [19]:
{s:a for s, a in enumerate(np.argmax(qt_, axis=1))} # enumerate just allows a counter to be automatically added when looping

{0: 3,
 1: 3,
 2: 0,
 3: 3,
 4: 0,
 5: 0,
 6: 0,
 7: 0,
 8: 3,
 9: 1,
 10: 0,
 11: 0,
 12: 0,
 13: 2,
 14: 1,
 15: 0}

In [20]:
for s, a in enumerate(np.argmax(qt_, axis=1)):
    print(s,a)

0 3
1 3
2 0
3 3
4 0
5 0
6 0
7 0
8 3
9 1
10 0
11 0
12 0
13 2
14 1
15 0


In [21]:
policy_evaluation(new_pi, P, gamma = 1.0, theta=1e-10)

array([0.29787234, 0.29787234, 0.29787234, 0.29787234, 0.40425532,
       0.        , 0.29787234, 0.        , 0.5106383 , 0.61702128,
       0.59574468, 0.        , 0.        , 0.74468085, 0.87234043,
       0.        ])

In [48]:
new_pi

<function __main__.policy_improvement.<locals>.<lambda>(s)>

In [43]:
#random_actions = np.random.choice(tuple(P[0].keys()), len(P))
def policy_iteration(P, gamma=1.0, theta=1e-10):
    random_actions = np.random.choice(tuple(P[0].keys()), len(P))#generate random policy for each state
    
    policy_ = lambda s : {s:a for s, a in enumerate(random_actions)}[s] #create dictionary func taking key as arg return value/action
    print(type(policy_(0)))
    while True:
        old_pi = {s:policy_(s) for s in range(len(P))}
        V  = policy_evaluation(policy_, P, gamma, theta)
        
        policy_,_ = policy_improvement(V, P, gamma) #a 'new improved' policy is created based off old state value func following policy pie
        
        print(type(policy_(0)))
        if old_pi == {s:policy_(s) for s in range(len(P))}:
            break
    return V, policy_,_
    

In [44]:
policy_iteration(P, gamma=1.0, theta=1e-10)

<class 'numpy.int32'>
(16, 4)
<class 'numpy.int64'>
(16, 4)
<class 'numpy.int64'>
(16, 4)
<class 'numpy.int64'>
(16, 4)
<class 'numpy.int64'>
(16, 4)
<class 'numpy.int64'>
(16, 4)
<class 'numpy.int64'>
(16, 4)
<class 'numpy.int64'>


(array([0.82352941, 0.82352941, 0.82352941, 0.82352941, 0.82352941,
        0.        , 0.52941176, 0.        , 0.82352941, 0.82352941,
        0.76470588, 0.        , 0.        , 0.88235294, 0.94117647,
        0.        ]),
 <function __main__.policy_improvement.<locals>.<lambda>(s)>,
 array([[0.82352941, 0.82352941, 0.82352941, 0.82352941],
        [0.54901961, 0.54901961, 0.54901961, 0.82352941],
        [0.72549019, 0.72549019, 0.72549019, 0.82352941],
        [0.54901961, 0.54901961, 0.54901961, 0.82352941],
        [0.82352941, 0.54901961, 0.54901961, 0.54901961],
        [0.        , 0.        , 0.        , 0.        ],
        [0.52941176, 0.25490196, 0.52941176, 0.2745098 ],
        [0.        , 0.        , 0.        , 0.        ],
        [0.54901961, 0.54901961, 0.54901961, 0.82352941],
        [0.56862745, 0.82352941, 0.54901961, 0.52941176],
        [0.76470588, 0.58823529, 0.49019608, 0.45098039],
        [0.        , 0.        , 0.        , 0.        ],
        [0.     

In [51]:
#random_actions(no) - this is VALUE ITERATION. no policy is explicitly described - greedy actions are taken and are used
#to update value of a state
def value_iteration(P, gamma = 1.0, theta = 1e-10):
    V = np.zeros(len(P))
    
    while True:
        Q_table = np.zeros((len(P), len(P[0])))
        
        for s in range(len(P)):
            for a in range(len(P[0])):
                for prob, next_state, reward, done in P[s][a]:
                    Q_table[s][a] = prob * (reward + gamma*V[next_state]*(not done))
        
        if np.max(np.abs(V - np.max(Q_table, axis = 1))) < theta:
            break
        
        V = np.max(Q_table, axis = 1)
    
    policy_ = lambda s : {s:a for s, a in enumerate(np.argmax(Q_table, axis=1))}[s]
    
    return V, policy_

In [52]:
value_iteration(P, gamma = 1.0, theta = 1e-10)

(array([0.00137174, 0.00411523, 0.01234568, 0.00411523, 0.00411523,
        0.        , 0.03703704, 0.        , 0.01234568, 0.03703704,
        0.11111111, 0.        , 0.        , 0.11111111, 0.33333333,
        0.        ]),
 <function __main__.value_iteration.<locals>.<lambda>(s)>)

In [45]:
Q_table = np.zeros((len(P), len(P[0])))

In [47]:
Q_table.shape

(16, 4)

In [48]:
V = np.zeros(P)

TypeError: expected sequence object with len >= 0 or a single integer